In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from tqdm.auto import tqdm
import plotly.express as px
import ast

In [ ]:
publishers = [
    "HarperCollins",
    "Little, Brown and Company",
    "Tor Books",
    "Disney Hyperion Books (Disney Publishing Group)",
    "Oxford World's Classics",
    "Doubleday (Knopf Doubleday Publishing Group)",
    "BLOOMSBURY PUBLISHING",
]

In [2]:
book_data = pd.read_csv(r"C:\Users\USER\OneDrive\桌面\C2L2\books_data.csv")

In [3]:
data_with_price_all_books = pd.read_csv(r"data_book_price\Best_Sellers_top100_1dep_other_info.csv", encoding="latin-1")

In [4]:
data_with_price_all_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831 entries, 0 to 6830
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   all_rank_list    6768 non-null   object
 1   format           6831 non-null   object
 2   before_discount  6831 non-null   object
 3   new_price        6831 non-null   object
 4   title            6831 non-null   object
 5   scrape_time      6831 non-null   object
 6   other_list       6811 non-null   object
dtypes: object(7)
memory usage: 373.7+ KB


In [5]:
book_data_valid = book_data[book_data["title"].isin(data_with_price_all_books["title"])]
book_price_data_valid = data_with_price_all_books[data_with_price_all_books["title"].isin(book_data_valid["title"])]
book_data_valid.info()
book_price_data_valid.info()
book_data_valid.to_excel("book_data_valid.xlsx", index=False)
book_price_data_valid.to_excel("book_price_data_valid.xlsx", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 2098 to 124935
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    65 non-null     object 
 1   ddc_sort                 22 non-null     object 
 2   ratings_average          21 non-null     float64
 3   ebook_count_i            65 non-null     int64  
 4   edition_count            65 non-null     int64  
 5   key                      65 non-null     object 
 6   publisher                65 non-null     object 
 7   publish_year             65 non-null     object 
 8   type                     65 non-null     object 
 9   isbn                     65 non-null     object 
 10  author_name              65 non-null     object 
 11  ratings_count_1          21 non-null     float64
 12  ratings_count_2          21 non-null     float64
 13  author_alternative_name  20 non-null     object 
 14  seed                 

In [6]:
valid_formats = ["Hardcover", "Paperback", "Kindle", "Audiobook"]


def format_check(x):
    for format in valid_formats:
        if format in str(x):
            return format
    return "other"


book_price_data_valid_format = book_price_data_valid["format"].apply(format_check)
book_price_data_valid["format"] = book_price_data_valid_format
book_price_data_valid.to_excel("book_price_data_valid.xlsx", index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_6728\2636834305.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_price_data_valid["format"] = book_price_data_valid_format


In [7]:
def get_price(x: str):
    try:
        return float(str(x).split("$")[-1])
    except:
        return np.nan


book_price_data_valid_before_discount = book_price_data_valid["before_discount"].apply(get_price)
book_price_data_valid["before_discount"] = book_price_data_valid_before_discount
book_price_data_valid_new_price = book_price_data_valid["new_price"].apply(get_price)
book_price_data_valid["new_price"] = book_price_data_valid_new_price
book_price_data_valid.to_excel("book_price_data_valid - price modified.xlsx", index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_6728\2566127772.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_price_data_valid["before_discount"] = book_price_data_valid_before_discount
C:\Users\USER\AppData\Local\Temp\ipykernel_6728\2566127772.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_price_data_valid["new_price"] = book_price_data_valid_new_price


In [8]:
valid_formats = ["Hardcover", "Paperback", "Kindle", "Audiobook"]

for format in valid_formats:
    format_price_data = book_price_data_valid[book_price_data_valid["format"] == format]
    # 將 book_data_valid 和 format_price_data 合併，並將 before_discount 值賦予到新的欄位
    book_data_valid = book_data_valid.merge(format_price_data[["title", "before_discount"]], on="title", how="left")
    book_data_valid.rename(columns={"before_discount": f"{format} Original Price"}, inplace=True)

for format in valid_formats:
    format_price_data = book_price_data_valid[book_price_data_valid["format"] == format]
    # 將 book_data_valid 和 format_price_data 合併，並將 before_discount 值賦予到新的欄位
    book_data_valid = book_data_valid.merge(format_price_data[["title", "new_price"]], on="title", how="left")
    book_data_valid.rename(columns={"new_price": f"{format} Selling Price"}, inplace=True)

for format in valid_formats:
    book_data_valid[f"{format} Discount Ratio"] = book_data_valid[f"{format} Selling Price"] / book_data_valid[f"{format} Original Price"]

In [23]:
book_data_valid["last_publish_year"] = book_data_valid["publish_year"].apply(lambda x: sorted(ast.literal_eval(x), reverse=True)[0])

In [32]:
book_data_valid = book_data_valid.merge(book_price_data_valid[["title", "all_rank_list"]], on="title", how="left")

In [38]:
book_data_valid["all_rank_list"] = book_data_valid["all_rank_list"].apply(lambda x: int(x[1:].replace(",", "")) if "#" in str(x) else np.nan)

In [35]:
book_data_valid.to_excel("book_data_valid.xlsx", index=False)
book_data_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 255
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     256 non-null    object 
 1   ddc_sort                  96 non-null     object 
 2   ratings_average           92 non-null     float64
 3   ebook_count_i             256 non-null    int64  
 4   edition_count             256 non-null    int64  
 5   key                       256 non-null    object 
 6   publisher                 256 non-null    object 
 7   publish_year              256 non-null    object 
 8   type                      256 non-null    object 
 9   isbn                      256 non-null    object 
 10  author_name               256 non-null    object 
 11  ratings_count_1           92 non-null     float64
 12  ratings_count_2           92 non-null     float64
 13  author_alternative_name   91 non-null     object 
 14  seed      

In [24]:
fig = px.scatter(book_data_valid, x="last_publish_year", y="Hardcover Discount Ratio", trendline="ols")
fig.show()

In [25]:
fig = px.scatter(book_data_valid, x="last_publish_year", y="Paperback Discount Ratio", trendline="ols")
fig.show()

In [27]:
fig = px.scatter(book_data_valid, x="Paperback Discount Ratio", y="Hardcover Discount Ratio", trendline="ols")
fig.show()

In [39]:
fig = px.scatter(book_data_valid, x="all_rank_list", y="Paperback Discount Ratio", trendline="ols")
fig.show()

In [40]:
fig = px.scatter(book_data_valid, x="all_rank_list", y="Hardcover Discount Ratio", trendline="ols")
fig.show()